# **Convolutional Neural Networks (CNN)**
ใน Notebook นี้เราจะมาลงมือทดลองสร้างและเทรน CNN เบื้องต้นด้วยไลบรารี่ Pytorch, Pytorch Lightning, และ FastAI รวมถึงการทำ Transfer Learning กัน


<img src="images/02_cnn_guide.png" width="600"/>

Convolutional Neural Network (CNN) ประกอบด้วย Convolutional layer, Pooling layer (ได้หลายขั้น) ตามด้วย Fully connected layer

In [ ]:
import torch

torch.cuda.is_available() # ตรวจสอบว่ามี CUDA หรือ GPU ที่สามารถใช้ได้หรือไม่
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# อ้างอิง: https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy/notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

train_dataset = torchvision.datasets.FashionMNIST(
    "./data", download=True, transform=transforms.Compose([transforms.ToTensor()])
)
test_dataset = torchvision.datasets.FashionMNIST(
    "./data", download=True, train=False,
    transform=transforms.Compose([transforms.ToTensor()])
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

In [ ]:
class FashionCNN(nn.Module):
    """Convolutional Neural Network เพื่อ Classify FashionMNIST"""
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Sequential(
            # บางครั้งเราเขียนแค่ nn.Conv2d(1, 32, 3, 1) 
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(in_features=64 * 7 * 7, out_features=600) # รู้ได้อย่างไรว่าเป็น 64*7*7 เราจะไปดูกันใน cell ถัดไป
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = torch.flatten(out, 1) # อาจจะใช้ view ก็ได้
        out = self.fc1(out)
        out = F.ReLU(out)
        out = self.fc2(out)
        out = F.ReLU(out)
        out = self.fc3(out)
        return out

### มาดู Dimension ของรูปภาพหลังผ่าน layer ต่างๆกัน

Dimension หลังจากผ่าน Convolutional layer สามารถคำนวณได้ด้วย (W - K - 2P)/S - 1 โดย
- W คือความกว้างหรือความยาวของรูปก่อนเข้า Convolutional layer
- K คือ Kernel size
- P คือ Padding จากภาพ
- S คือ Stride (ส่วนมากจะใช้ค่าเท่ากับ 1)


ส่วน MaxPool2d layer ทำการดึงข้อมูลที่มีค่าสูงที่สุดจาก stride ที่เรากำหนด โดยพื้นฐานส่วนมากเราจะใช้ kernel size = 2, stride = 2

มี Padding

In [ ]:
dummy_img = torch.rand(1,28,28)
print(f"Image size: {dummy_img.size()}")

# ทดลองสร้าง Convolutional layer ที่รับ input depth = 1, output = 32, kernel size = 3, padding = 1
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
maxpool1= nn.MaxPool2d(kernel_size=(2,2))
out1 = conv1(dummy_img)
print(f"Output of conv1: {out1.size()}") # (32, 28, 28)
out1 = maxpool1(out1)
print(f"Output of maxpool1: {out1.size()}") # (32, 14, 14)

conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
maxpool2= nn.MaxPool2d(kernel_size=(2,2))

out2 = conv2(out1)
print(f"Output of conv2: {out2.size()}") # (64, 14, 14)
out2 = maxpool2(out2)
print(f"Output of maxpool2: {out2.size()}") # (64, 7, 7)

# สังเกต dimension ของ output นี้ว่ามิติแรกเท่ากับจำนวน image filters ของ conv2
# และ มิติที่สองและสามมีค่าเท่ากับ (28, 28) หารด้วย 4 ทั้งสองมิติ หรือ (7,7) 
# เนื่องจากผ่านการ maxpool ด้วย kernel_size = (2,2) ถึง 2 ครั้งนั่นเอง

ไม่มี Padding

In [ ]:
dummy_img = torch.rand(1,28,28)
print(f"Image size: {dummy_img.size()}")

# ทดลองสร้าง Convolutional layer ที่รับ input depth = 1, output = 32, kernel size = 3
conv1 = nn.Conv2d(1, 32, kernel_size=3)
maxpool1= nn.MaxPool2d(kernel_size=(2,2))
out1 = conv1(dummy_img)
print(f"Output of conv1: {out1.size()}") # (32, 26, 26)
out1 = maxpool1(out1)
print(f"Output of maxpool1: {out1.size()}") # (32, 13, 13)

conv2 = nn.Conv2d(32, 64, kernel_size=3)
maxpool2= nn.MaxPool2d(kernel_size=(2,2))

out2 = conv2(out1)
print(f"Output of conv2: {out2.size()}") # (64, 11, 11)
out2 = maxpool2(out2)
print(f"Output of maxpool2: {out2.size()}") # (64, 5, 5)

# สังเกต dimension ของ output นี้ว่ามิติแรกเท่ากับจำนวน image filters ของ conv2
# และ มิติที่สองและสามมีค่าเท่ากับ (28, 28) หารด้วย 4 แล้วลบด้วย 2 ทั้งสองมิติ หรือ (5,5)
# เนื่องจากผ่านการ maxpool ด้วย kernel_size = (2,2) ถึง 2 ครั้ง (การหาร 4) 
# และลบด้วย 2 เนื่องจากเป็นส่วนขอบรูปที่ตกไปในการ filter แบบไม่เติม padding นั่นเอง

### **Train our FashionCNN**

เทรนโมเดล CNN โดยใช้ Loop การเรียนรู้ที่เราสร้างขึ้น จากนั้นเราทำการทดสอบโมเดล

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, n_batch = 100, device = "cuda"):
    size = len(dataloader.dataset)
    # Iterate over the dataloader
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Report loss every 100 batch
        if batch % n_batch == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn, device = "cuda"):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Prevent Pytorch from updating the gradients at the testing steps!
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

epochs = 10
learning_rate = 1e-3

net = FashionCNN().to(device)
loss_fn = nn.CrossEntropyLoss() # loss function for classification
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate) # Optimizer
scheduler = ReduceLROnPlateau(optimizer, "min")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device = device)
    val_loss = test_loop(test_dataloader, net, loss_fn, device = device)
    scheduler.step(val_loss)
print("Done!")

## **Transfer Learning ด้วย Pytorch**

- Transfer learning เป็นหนึ่งในเทคนิคของ Machine learning ที่นำโมเดลที่สร้างมาสำหรับงานประเภทหนึ่งถูกนำมาใช้กับงานอีกประเภทหนึ่ง
- หลักการโดยคร่าวๆของ Transfer learning คือการนำโมเดลที่ถูกเทรนมาแล้ว (Pre-trained model) มาทำการดัดแปลงพารามิเตอร์บางส่วนจากโมเดลเดิมและเทรนโมเดลสำหรับข้อมูลใหม่ของเรา


อ้างอิง: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#convnet-as-fixed-feature-extractor


``` py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

class TransferNet(nn.Module):
    def __init__(self, backbone, num_classes):
        super(TransferNet, self).__init__()
        self.model = backbone
        # "freeze" weights ของ backbone ไม่ให้เปลี่ยนแปลง
        for param in self.model.parameters():
            param.requires_grad = False
        # เปลี่ยน layer สุดท้ายของโมเดลที่เลือกใช้งาน
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.model(x)
```


**โหลดโมเดล ResNet18 จาก torchvision.models และส่งเข้า class TransferNet ของเรา**
``` py
backbone = torchvision.models.resnet18(pretrained=True)
net = TransferNet(backbone = backbone, num_classes = 2)
```

### **ทำการเทรนด้วย `train_loop()` และ `test_loop()` ที่ได้เขียนไว้**

``` py
from torch.optim.lr_scheduler import ReduceLROnPlateau

epochs = 50
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss() # loss function for classification
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate) # Optimizer
scheduler = ReduceLROnPlateau(optimizer, "min")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer)
    val_loss = test_loop(val_dataloader, net, loss_fn, optimizer)
    scheduler.step(val_loss)
print("Done!")
```

## **Transfer Learning ด้วย Pytorch Lightning**

นอกจากการเขียน Transfer learning ด้วย Pytorch นั้น เรายังสามารถใช้ไลบรารี่อื่นๆที่ทำให้การเทรนโมเดลของเราสั้นลงได้ เช่น Pytorch Lightning โดยวิธีการเขียนสามารถดูได้ตามด้านล่าง

**Install ไลบรารี่ pytorch_lightning**
``` py
!pip install pytorch_lightning
```


**Define โมเดลของเราโดย inherit จาก pl.LightningModule แทน nn.Module**
``` py
import pytorch_lightning as pl
import torch.nn.functional as F

class TransferNet(pl.LightningModule): 
    def __init__(self, backbone, num_classes, learning_rate):
        super(TransferNet, self).__init__()
        self.model = backbone
        self.learning_rate = learning_rate
        # "freeze" weights ของ backbone ไม่ให้เปลี่ยนแปลง
        for param in self.model.parameters():
            param.requires_grad = False
        # เปลี่ยน layer สุดท้ายของโมเดลที่เลือกใช้งาน
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

    # กำหนด loop การเทรนโมเดลภายใน class
    def training_step(self, batch, batch_idx):
        x, y = batch
        prediction = self(x)
        loss = F.cross_entropy(prediction, y)
        return loss

    # กำหนดเลือก optimizer ที่นี่เช่นเดียวกัน
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.learning_rate)
```

### **เทรนด้วย `pl.Trainer()`**

``` py
trainer = pl.Trainer(max_epochs=50, accelerator = "gpu")
backbone = torchvision.models.resnet18(pretrained=True)
net = TransferNet(
    backbone = backbone,
    num_classes = 2,
    learning_rate=1e-3) 

trainer.fit(net, train_dataloaders=train_dataloader)
```

## **Transfer Learning ด้วย FastAI**

``` py
# https://www.analyticsvidhya.com/blog/2021/05/training-state-of-the-art-deep-learning-models-with-fast-ai/

from fastai import *
from fastai.vision import *

# กำหนด folder ที่เก็บข้อมูลไว้ให้กับ ImageDataLoaders
# ภายใต้ path ที่กำหนดจะมี 2 folders ย่อย คือ training/ และ validation/ 
dls = ImageDataLoaders.from_folder(path=path, 
                                    train='training',
                                    valid='validation',
                                    valid_pct=0.2,
                                    shuffle=True)

# กำหนด backbone ที่เราต้องการใช้งานให้กับ cnn_learner เช่น resnet18
learner = cnn_learner(dls, 
                    resnet18, 
                    metrics=[accuracy, error_rate])

learner.fine_tune(4)
```

## **Augmentations ด้วย `torchvision.transforms`**

``` py
import torchvision.transforms as T

train_transform = T.Compose([
    T.Resize((256, 256)),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.RandomResizedCrop((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

train_data = AnimalDataset(train_filenames, train_labels, transform=train_transform)
val_data = AnimalDataset(val_filenames, val_labels, transform=val_transform)
```

# **มาทดลองการจำแนกรูปภาพกัน**

Taken from [AI builders' repository](https://github.com/ai-builders/curriculum/blob/main/notebooks/04v_classification_pytorch.ipynb)

**Authored by**: Titipat

In [ ]:
## upload your kaggle.json file to colab workspace first
## before running this cell

!mkdir /root/.kaggle
!cp kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# SOURCE: https://www.kaggle.com/competitions/dog-breed-identification/data

!kaggle competitions download -c dog-breed-identification
!unzip dog-breed-identification.zip -d data

In [ ]:
!pip install pytorch_lightning
!pip install --upgrade numpy # จำเป็นเพื่อป้องกัน errors จากจุดทศนิยม

In [ ]:
import os
import os.path as op
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# get all image paths
img_df = pd.DataFrame(glob("data/train/*.jpg"), columns=["path"])
img_df["id"] = img_df.path.map(lambda x: op.basename(x).replace(".jpg", ""))

# read label data
label_df = pd.read_csv("data/labels.csv")
train_df = img_df.merge(label_df, on="id")

In [ ]:
train_df.head()

In [ ]:
train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=3)

In [ ]:
print("Length of training set = {}, validation set = {}".format(len(train_df), len(validation_df)))

In [ ]:
root_dir = "data/dogdata/"
for df, f in zip([train_df, validation_df], ["train", "validation"]):
    for _, r in df.iterrows():
        # create subfolder if it doesn't exist
        d = op.join(root_dir, f, r.breed)
        if not op.exists(d):
            os.makedirs(d)
        shutil.copy(r.path, op.join(root_dir, f, r.breed, f"{r.id}.jpg"))

### **Image classification ด้วย Pytorch Lightning**

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
train_transform = T.Compose([
    T.Resize((256, 256)),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.RandomResizedCrop((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

In [ ]:
train_dataset = datasets.ImageFolder("data/dogdata/train/", transform=train_transform)
val_dataset = datasets.ImageFolder("data/dogdata/validation/", transform=val_transform)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
classes = train_data.classes
n_classes = len(classes)

In [ ]:
class DogResNet(pl.LightningModule):
    def __init__(self, n_classes=120):
        super(DogResNet, self).__init__()
        
        # จำนวนของพันธุ์น้องหมา (120)
        self.n_classes = n_classes

        # ใช้สถาปัตยกรรม resnet34; เปลี่ยน layer สุดท้าย
        self.backbone = models.resnet34(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False
        # เปลี่ยน fc layer เป็น output ขนาด 120
        self.backbone.fc = torch.nn.Linear(self.backbone.fc.in_features, n_classes)
        
        self.entropy_loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy()

    def forward(self, x):
        preds = self.backbone(x)
        return preds

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        acc = self.accuracy(y_pred, y)
        
        # log metrics ที่สำคัญไว้เพื่อการวิเคราะห์ในภายหลัง
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return {"loss": loss, "accuracy": acc}

    @torch.no_grad()
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        y_pred = torch.argmax(logits, dim=1)
        acc = self.accuracy(y_pred, y)
        
        # log metrics ที่สำคัญไว้เพื่อการวิเคราะห์ในภายหลัง
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return {"loss": loss, "accuracy": acc}

    @torch.no_grad()
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        y_pred = torch.argmax(logits, dim=1)
        acc = self.accuracy(y_pred, y)
        return {"loss": loss, "accuracy": acc}

    def configure_optimizers(self):
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        return {
            "optimizer": self.optimizer,
            "monitor": "val_loss",
        }

In [ ]:
model = DogResNet(n_classes=n_classes)

In [ ]:
# callback เพื่อให้ Trainer เซฟโมเดลไว้ในไฟล์ checkpoint เมื่อผลลัพธ์ val_loss ลดต่ำกว่าที่เคย
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints/dogbreed/",
    filename="resnet18--{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

In [ ]:
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

## **Image Classification ด้วย FastAI**

In [ ]:
!pip install fastai --upgrade

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/05/training-state-of-the-art-deep-learning-models-with-fast-ai/

from fastai import *
from fastai.vision.all import *

# กำหนด folder ที่เก็บข้อมูลไว้ให้กับ ImageDataLoaders
# ภายใต้ path ที่กำหนดจะมี 2 folders ย่อย คือ training/ และ validation/ 
dls = ImageDataLoaders.from_folder(path="data/dogdata", 
                                    train="train",
                                    valid="validation",
                                    item_tfms=Resize(224),
                                    bs = 64,
                                    shuffle=True)

# กำหนด backbone ที่เราต้องการใช้งานให้กับ cnn_learner เช่น resnet18
learner = cnn_learner(dls,
                    resnet18,
                    metrics=[accuracy, error_rate])

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(10, 1e-3)

In [ ]:
learner.show_results()

## **แบบทดสอบ**

1. จงสร้าง Convolutional layer ที่รับ input
``` py
dummy_input = torch.rand(3, 28, 28)
```
โดยมี filters จำนวน 64 filters, `kernel_size = 3` และ มี padding

2. จงคำนวณขนาด (dimensions) ของ output เมื่อผ่าน convolutional layer ดังกล่าว

3. จงสร้าง Max pooling layer ที่รับ output ของ convolutional layer ในข้อก่อนหน้าเป็น input โดยมีขนาด `kernel_size = 2` และบอกขนาด (dimensions) ของ output เมื่อผ่าน Max pooling layer ดังกล่าว